In [1]:
import os
import sys
src_path = os.path.abspath('../src')
sys.path.append(src_path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np

from src import data
from src import portfolio
from src import efficient_frontier
from src import plot
from src import capm
import yfinance as yf
from scipy.optimize import Bounds, LinearConstraint
import statsmodels.api as smf

In [2]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1100
max_esg_score = 2000
df = pd.read_excel(r"../data/ESG_DATA_S&P500.xlsx")

dates = ['2021-01-01','2022-01-01']
start_year = dates[0]
end_year = dates[1]

Bounds = Bounds(0,1) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.02

In [3]:
esg_data = data.esg_score_weight(df,weights,min_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data) #Gives us price data and esg data to use in future functions
prices = prices[['MMM','AOS','ABT','ACN','ADM','ADP']]
esgdata = esgdata[['MMM_weighted','AOS_weighted','ABT_weighted','ACN_weighted','ADM_weighted','ADP_weighted']]

pct_returns = data.pct_returns_from_prices(prices) #Gives us the returns in pct for our stocks
parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds, sharpe_type,dates[0],dates[1], Wanted_return, maximum_risk) #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters)
sp500 = data.data_for_beta(prices,['SPY'],dates)
pct_returns_sp500 = data.pct_returns_from_prices(sp500)
market_name = 'SPY'


Min. Risk = 14.118% => Return: 29.436%  Sharpe Ratio = 2.09
Max. Sharpe Ratio = 3.19 => Return: 50.51%  Risk: 15.837%


In [4]:
print(sp500)
betas_of_portfolio = capm.calculate_portfolio_beta(pct_returns_sp500,prices,weights_of_each_portfolio,market_name)
betas_of_portfolio

#1.3142


                                  MMM        AOS         ABT         ACN  \
Date                                                                       
2021-01-01 00:00:00-05:00  175.660004  54.299999  123.589996  241.919998   
2021-02-01 00:00:00-05:00  175.059998  59.369999  119.779999  250.899994   
2021-03-01 00:00:00-05:00  192.679993  67.610001  119.839996  276.250000   
2021-04-01 00:00:00-04:00  197.139999  67.750000  120.080002  289.970001   
2021-05-01 00:00:00-04:00  203.039993  71.070000  116.650002  282.160004   
2021-06-01 00:00:00-04:00  198.630005  72.059998  115.930000  294.790009   
2021-07-01 00:00:00-04:00  197.940002  70.330002  120.980003  317.679993   
2021-08-01 00:00:00-04:00  194.740005  72.720001  126.370003  336.559998   
2021-09-01 00:00:00-04:00  175.419998  61.070000  118.129997  319.920013   
2021-10-01 00:00:00-04:00  178.679993  73.070000  128.889999  358.790009   
2021-11-01 00:00:00-04:00  170.039993  79.050003  125.769997  357.399994   
2021-12-01 0

1.3142198303380912

In [5]:

capm.capm_calc(pct_returns_sp500['SPY'].mean(),risk_free_rate,betas_of_portfolio)



Excpected return on investment is 128.8403793484515%


1.2884037934845152

In [6]:
capm.calculate_portfolio_beta_ols(pct_returns_sp500,prices,weights_of_each_portfolio)

UFuncTypeError: ufunc 'maximum' did not contain a loop with signature matching types (dtype('<U3'), dtype('<U3')) -> None